# Charging Model

## initialize set
- import modules
- set problems
- set idnex
- define variables
- read data from files
- define constants


In [81]:
# import Modules
import xpress as xp
import numpy as np
import pandas as pd
# use professional edition xpress package!!!

In [2]:
# Create a problem called myproblem

prob = xp.problem(name='charging')

In [3]:
# Set index
number_of_types = 3 # i: 1..3
types = range(number_of_types)

number_of_years = 4 # Time t: 1..4 0..4
years = range(number_of_years)
number_of_years2 = number_of_years + 1
years2 = range(number_of_years2)

number_of_grids = 434 # grid g: 1..434
grids = range(number_of_grids)


In [4]:
# Define our decision variable as a numpy array
x1 = np.array([xp.var( name='x1_{0}_{1}_{2}'.format(i+1,g+1,t+1),vartype = xp.integer)
                    for i in types for g in grids for t in years], dtype=xp.npvar).reshape(number_of_types,number_of_grids,number_of_years)
x2 = np.array([xp.var( name='x2_{0}_{1}_{2}'.format(i+1,g+1,t+1),vartype = xp.integer)
                    for i in types for g in grids for t in years], dtype=xp.npvar).reshape(number_of_types,number_of_grids,number_of_years)
x = np.array([xp.var( name='x_{0}_{1}_{2}'.format(i+1,g+1,t),vartype = xp.integer)
                    for i in types for g in grids for t in range(number_of_years2)], dtype=xp.npvar).reshape(number_of_types,number_of_grids,number_of_years2)
# x = x1 + x2
z = np.array([xp.var( name='z_{0}_{1}'.format(g+1,t+1),vartype = xp.binary)
                    for g in grids for t in years], dtype=xp.npvar).reshape(number_of_grids,number_of_years)

y = np.array([xp.var( name='y_{0}_{1}'.format(g+1,t+1),vartype = xp.integer)
                    for g in grids for t in years], dtype=xp.npvar).reshape(number_of_grids,number_of_years)

prob.addVariable(x1,x2,x,y,z)

In [5]:
# read data from files
pot_char_poi = pd.read_excel('Project_data/Potential_charging_points.xlsx')
intere_poi = pd.read_excel('Project_data/Interest _points.xlsx')
demand = pd.read_excel('Project_data/Demand_data.xlsx')
char_poi =  pd.read_excel('Project_data/Charging_points.xlsx')


In [6]:
#  Constant
c = np.array(types) # building cost for charging point of type i
c = [1,1,1] #!!!Notice: can define by ourselves

# the reduced building cost for charging point of type i
r = np.array(types) 
r = [0.8,0.8,0.8] #!!!Notice: can define by ourselves

# the maximal number of location in the grid square g
B = np.array(grids) 
for i in B:
    i = 3 #!!!Notice: can define by ourselves
    
# limitation of charger's number in every location
A = 5 #!!!Notice: can define by ourselves

# A*B_{g}:the maximal number of charging points in the grid square g, where 

# the lower bound of proportion for type i charging point
L = np.array(types)
L = [0.3,0.3,0.3] #!!!Notice: can define by ourselves

# the Upper bound of proportion for type i charging point
U = np.array(types) 
U = [0.7,0.7,0.7] #!!!Notice: can define by ourselves

# maximal number of station points in the centre of city.
SP = 100 #!!!Notice: can define by ourselves

# the satisfaction proportion of for the total demand
TS = 0.9 #!!!Notice: can define by ourselves

# the proportion of the least demand provided by neighbor for 1 grid.
MS = 0.8 #!!!Notice: can define by ourselves

# define which grids belong to center by "distance to center"
DCenter = 1000 #!!!Notice: can define by ourselves

# demand for the grid g in year t
d = np.array((demand['Demand_0'],demand['Demand_1'],demand['Demand_2'],demand['Demand_3']))

# the minimum demand type i can provide per year
Pi1 = np.array(years) 
Pi1 = [2000,4000,30000]

# the maximum demand type i can provide per year 
Pi = np.array(years) 
Pi = [3500,5200,55000]

# Pg: the number of potential locations in the grid square
pg = (pot_char_poi['grid number'].value_counts())
Pg = np.zeros(number_of_grids) 
for i in pg.index:
    Pg[i-1] = pg[i]
    

## Add Constraints

In [7]:
# constrain 0
# relastion between variables and exist constants
char_poi['Type'].replace('Slow',0,inplace = True)
char_poi['Type'].replace('Fast',1,inplace = True)
char_poi['Type'].replace('Rapid',2,inplace = True)
exist_charger = pd.DataFrame({'Type':char_poi['Type'],'Grid':char_poi['grid number']})


for i in types:
    for g in grids:
        prob.addConstraint( x[i,g,0] == ((exist_charger.Type == i) & (exist_charger.Grid ==g)).sum())


In [8]:
# constrain 1 
# the number of chargers cannot be more than limitation in t years in g grids

prob.addConstraint(y[g,t] <= A*B[g] for g in grids for t in years)

In [9]:
# constrian 2
# the limitation of charger number in city center
distance = demand['Distance from Centre'].copy()
distance.sort_values(inplace = True)
center = distance[distance <= DCenter]
center_grid = np.array(center.index)


prob.addConstraint((xp.Sum(y[int(g),t] for g in center_grid) <= SP) for t in years)  


In [10]:
# constrain 3
# multi chargers' types
for i in types:
    for t in years2:
        if t>0:
            prob.addConstraint(L[i]*xp.Sum(x[i1,g,t1] for i1 in types for g in grids for t1 in years2 if t1<=t) <= 
                          xp.Sum(x[i,g,t1] for g in grids for t1 in years2 if t1 <=t))
            prob.addConstraint(U[i]*xp.Sum(x[i1,g,t1] for i1 in types for g in grids for t1 in years2 if t1<=t) >= 
                          xp.Sum(x[i,g,t1] for g in grids for t1 in years2 if t1<=t))


In [11]:
# constrain 4
# the proportion of total demand at least should be satisfied(TS)

TotalDemand = np.array([np.sum(d[:][0]),np.sum(d[:][1]),np.sum(d[:][2]),np.sum(d[:][3])])
prob.addConstraint(2*xp.Sum(Pi1[i]*xp.Sum(x[i,g,t1] for g in grids) 
                          for i in types for t1 in years2 if t1<=t)>=TS*TotalDemand[t-1] for t in years2 if t>=1)


In [12]:
# constraint 5
# the proportion of 1 grid's demand satisfied by itselves and neighbors
neighbors = demand['NEIGHBORS']
for g in grids:
    neighbor = np.array(neighbors[g].strip('[]').split(','),dtype = 'int')
    neighbor = np.insert(neighbor,0,[int(demand['Ref'][g])])
    prob.addConstraint(xp.Sum(Pi1[i]*xp.Sum(x[i,n-1,t1] for n in neighbor) for i in types for t1 in years2 if t1<=t)
                      >= MS*d[t-1][g] for t in years2 if t >=1)


In [13]:
# constraint 6
# x = x1 + x2
prob.addConstraint(x[i,g,t+1] == x1[i,g,t] + x2[i,g,t] for i in types for g in grids for t in years)

In [14]:
# constraint 7
# relation between x and y 
for g in grids:
    prob.addConstraint(y[g,0] == xp.Sum((x[i,g,0]+x[i,g,1]) for i in types))
    prob.addConstraint(y[g,1] == xp.Sum((x[i,g,0]+x[i,g,1] + x[i,g,2]) for i in types))
    prob.addConstraint(y[g,2] == xp.Sum((x[i,g,0]+x[i,g,1] + x[i,g,2] + x[i,g,3]) for i in types))
    prob.addConstraint(y[g,3] == xp.Sum((x[i,g,0]+x[i,g,1] + x[i,g,2] + x[i,g,3] + x[i,g,4]) for i in types))


In [15]:
# constraint 8
# calculate reduced cost
M = 1000
# M = A*B[0]
        
prob.addConstraint((1-z[g,t]) <= y[g,t] for g in grids for t in years)
prob.addConstraint(y[g,t] <= (1-z[g,t])*M for g in grids for t in years)
prob.addConstraint(x1[i,g,t] <= M*z[g,t] for i in types for g in grids for t in years)
prob.addConstraint(x2[i,g,t] <= M*(1-z[g,t]) for i in types for g in grids for t in years)


In [16]:
# constraint 9
# binary variables' definition z \in {0,1}
# written in prebious module

In [17]:
# constraint 10
# >= 0
prob.addConstraint(x1[i,g,t] >= 0 for i in types for g in grids for t in years)
prob.addConstraint(x2[i,g,t] >= 0 for i in types for g in grids for t in years)
prob.addConstraint(y[g,t] >= 0 for g in grids for t in years)

In [18]:
# constraint 11
# chargers have to be produced in potential locations
for g in grids:
    prob.addConstraint(x[i,g,t+1] <= Pg[g]*A for i in types for t in years)


## Solving Problem

In [19]:
# Objective function
prob.setObjective(xp.Sum(c[i]*x1[i,g,t] for i in types for g in grids for t in years) 
                  + xp.Sum(r[i] * x2[i,g,t] for i in types for g in grids for t in years),
                 sense = xp.minimize)

In [20]:
prob.write("problem","lp")

In [21]:
prob.solve()

FICO Xpress v8.13.7, Hyper, solve started 14:45:14, Nov 28, 2022
Heap usage: 25MB (peak 25MB, 9797KB system)
Minimizing MILP charging using up to 8 threads, with these control settings:
OUTPUTLOG = 1
Original problem has:
     42998 rows        20398 cols       364286 elements     20398 globals
Presolved problem has:
      1665 rows         1379 cols        16235 elements      1379 globals
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 23MB (peak 56MB, 9797KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 3.00e-01,  6.00e+04] / [ 1.95e-03,  1.99e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  9.40e+05] / [ 1.00e+00,  5.10e+02]
  Objective      [min,max] : [ 8.00e-01,  1.00e+00] / [ 8.00e-01,  1.00e+00]
Autoscaling applied standard scaling

Symmetric problem: generators: 16, support set: 384
 Number of orbits: 128, largest orbit: 7
 Row orbits: 160, row support: 480
Will try to keep branch and bound

## Result

In [38]:
print("the objective value is: ",prob.getObjVal())

the objective value is:  32.80000000000001


In [23]:
X = prob.getSolution(x)
Type1 = X[0]
Type2 = X[1]
Type3 = X[2]

In [77]:
print("Slow charger :")
type1 = np.nonzero(Type1)
for i in range(len(type1[0])):
    print("set charger in grid ",type1[0][i],' in year ',type1[1][i])
print('if year equal to 0, the charger has been set previously.')

Slow charger :
set charger in grid  76  in year  0
set charger in grid  91  in year  0
set charger in grid  117  in year  0
set charger in grid  129  in year  0
set charger in grid  146  in year  0
set charger in grid  147  in year  0
set charger in grid  150  in year  0
set charger in grid  159  in year  0
set charger in grid  160  in year  0
set charger in grid  163  in year  0
set charger in grid  173  in year  0
set charger in grid  189  in year  0
set charger in grid  190  in year  0
set charger in grid  199  in year  0
set charger in grid  200  in year  0
set charger in grid  202  in year  0
set charger in grid  205  in year  0
set charger in grid  214  in year  0
set charger in grid  228  in year  0
set charger in grid  259  in year  0
set charger in grid  263  in year  0
set charger in grid  274  in year  0
set charger in grid  288  in year  0
set charger in grid  300  in year  0
set charger in grid  301  in year  0
set charger in grid  304  in year  0
set charger in grid  314 

In [79]:
print("Fast charger :")
type2 = np.nonzero(Type2)
for i in range(len(type2[0])):
    print("set charger in grid ",type2[0][i],' in year ',type2[1][i])
print('if year equal to 0, the charger has been set previously.')

Fast charger :
set charger in grid  59  in year  0
set charger in grid  76  in year  0
set charger in grid  89  in year  1
set charger in grid  90  in year  1
set charger in grid  100  in year  1
set charger in grid  101  in year  1
set charger in grid  104  in year  1
set charger in grid  106  in year  1
set charger in grid  108  in year  1
set charger in grid  115  in year  1
set charger in grid  143  in year  1
set charger in grid  144  in year  0
set charger in grid  145  in year  1
set charger in grid  161  in year  1
set charger in grid  175  in year  1
set charger in grid  176  in year  1
set charger in grid  185  in year  0
set charger in grid  199  in year  0
set charger in grid  200  in year  0
set charger in grid  214  in year  0
set charger in grid  216  in year  0
set charger in grid  229  in year  1
set charger in grid  241  in year  1
set charger in grid  289  in year  1
set charger in grid  345  in year  0
set charger in grid  369  in year  0
if year equal to 0, the cha

In [80]:
print("Rapid charger :")
type3 = np.nonzero(Type3)
for i in range(len(type3[0])):
    print("set charger in grid ",type3[0][i],' in year ',type3[1][i])
print('if year equal to 0, the charger has been set previously.')

Rapid charger :
set charger in grid  89  in year  1
set charger in grid  103  in year  0
set charger in grid  118  in year  1
set charger in grid  120  in year  1
set charger in grid  134  in year  0
set charger in grid  142  in year  1
set charger in grid  183  in year  1
set charger in grid  184  in year  1
set charger in grid  186  in year  0
set charger in grid  187  in year  1
set charger in grid  188  in year  1
set charger in grid  190  in year  0
set charger in grid  197  in year  1
set charger in grid  199  in year  0
set charger in grid  216  in year  0
set charger in grid  245  in year  1
set charger in grid  246  in year  1
set charger in grid  263  in year  0
set charger in grid  271  in year  1
set charger in grid  285  in year  1
set charger in grid  287  in year  1
set charger in grid  290  in year  1
set charger in grid  298  in year  1
set charger in grid  341  in year  1
set charger in grid  342  in year  1
set charger in grid  354  in year  1
set charger in grid  35